In [ ]:
import pandas as pd
import numpy as np
from fake_useragent import UserAgent
from lxml import etree

# 哪些地址参与了集卡
- NFT汇集地址：0x77C207739ABeB3dd3380352f52C607A08e57D837

In [93]:
import requests
import json
from collections import defaultdict

# 配置
API_KEY = "E8SK4VKKAXNR98Q93G7HMUGRCVAIXKXWDF"  # 替换为你的BscScan API密钥
ADDRESS = "0x77C207739ABeB3dd3380352f52C607A08e57D837" # NFT汇集地址
API_URL = "https://api.etherscan.io/v2/api"

def query(params):
    try:
        response = requests.get(API_URL, params=params)
        response.raise_for_status()
        data = response.json()
        
        if data["status"] == "1" and data["message"] == "OK":
            return data["result"]
        elif data["status"] == "0":
            return []
        else:
            print(f"API错误: {data['message']}")
            return []
    except requests.RequestException as e:
        print(f"请求失败: {e}")
        return []

In [ ]:
def get_transactions(address):
    params = {
    "chainid": "56",  # BSC主网
    "module": "account",
    "action": "txlist",  
    "address": address,
    "startblock": 0,
    "endblock": 99999999,
    "sort": "asc",
    "page":1,
    "offset": 100, 
    "apikey": API_KEY,
}
    data = query(params)
    if not data:
        print("无交易记录或获取失败")
        return []
    return data


In [6]:
# 获取指定合约的已发行代币数量
def get_token_supply(chainid, address):
    # 基于合约 totalSupply() 函数的调用。
    # 如果合约没有实现或暴露 totalSupply()，这个接口就没法返回真实的发行量。
    # 这时，tokensupply 接口返回 0 或错误，不能反映真实已经 mint 出来的 NFT 数量。
    params = {
    "chainid": chainid,  # BSC主网
    "module": "stats",
    "action": "tokensupply",  
    "contractaddress": address,
    "apikey": API_KEY,
}
    data = query(params)
    if not data:
        print("无交易记录或获取失败")
        return []
    return data


In [ ]:
def get_NFT_Transfers(address):
    params = {
    "chainid": "56",  # BSC主网
    "module": "account",
    "action": "tokennfttx",  # 获取BEP-721/BEP-1155转账记录
    "address": address,
    "startblock": 0,
    "endblock": 99999999,
    "sort": "asc",
    "apikey": API_KEY,
    "page":0, # 0是全部页面
    "offset": 100 
}
    data = query(params)
    if not data:
        print("无交易记录或获取失败")
        return []
    return data


In [35]:
NFT_Transfers = get_NFT_Transfers(address=ADDRESS)

In [36]:
df_NFT_Transfers = pd.DataFrame(NFT_Transfers)

In [37]:
df_NFT_Transfers

,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,tokenID,tokenName,...,tokenDecimal,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,input,methodId,functionName,confirmations
0,47252620,1741339463,0x3853fa818918c9e94701372bfa5778db46d56e5e7ad8...,155,0xe67af1a9d0bc13464901556c1b4afac0a9c255843d8f...,0xccddd8403bf28ccdedce93c1985032281670cf48,0x236489bdbf9f4f297a214d365be8f7238fc247ae,0x77c207739abeb3dd3380352f52c607a08e57d837,99301,KLK Genesis,...,0,1,7768662,3000000000,4319495,4347087,deprecated,0x89d56c84,"transferERC721s(address to, address[] tokens, ...",8586612
1,47252620,1741339463,0x3853fa818918c9e94701372bfa5778db46d56e5e7ad8...,155,0xe67af1a9d0bc13464901556c1b4afac0a9c255843d8f...,0xccddd8403bf28ccdedce93c1985032281670cf48,0x236489bdbf9f4f297a214d365be8f7238fc247ae,0x77c207739abeb3dd3380352f52c607a08e57d837,99170,KLK Genesis,...,0,1,7768662,3000000000,4319495,4347087,deprecated,0x89d56c84,"transferERC721s(address to, address[] tokens, ...",8586612
2,47252620,1741339463,0x3853fa818918c9e94701372bfa5778db46d56e5e7ad8...,155,0xe67af1a9d0bc13464901556c1b4afac0a9c255843d8f...,0xccddd8403bf28ccdedce93c1985032281670cf48,0x236489bdbf9f4f297a214d365be8f7238fc247ae,0x77c207739abeb3dd3380352f52c607a08e57d837,20270,KLK Genesis,...,0,1,7768662,3000000000,4319495,4347087,deprecated,0x89d56c84,"transferERC721s(address to, address[] tokens, ...",8586612
3,47252620,1741339463,0x3853fa818918c9e94701372bfa5778db46d56e5e7ad8...,155,0xe67af1a9d0bc13464901556c1b4afac0a9c255843d8f...,0xccddd8403bf28ccdedce93c1985032281670cf48,0x236489bdbf9f4f297a214d365be8f7238fc247ae,0x77c207739abeb3dd3380352f52c607a08e57d837,24012,KLK Genesis,...,0,1,7768662,3000000000,4319495,4347087,deprecated,0x89d56c84,"transferERC721s(address to, address[] tokens, ...",8586612
4,47252620,1741339463,0x3853fa818918c9e94701372bfa5778db46d56e5e7ad8...,155,0xe67af1a9d0bc13464901556c1b4afac0a9c255843d8f...,0xccddd8403bf28ccdedce93c1985032281670cf48,0x236489bdbf9f4f297a214d365be8f7238fc247ae,0x77c207739abeb3dd3380352f52c607a08e57d837,21999,KLK Genesis,...,0,1,7768662,3000000000,4319495,4347087,deprecated,0x89d56c84,"transferERC721s(address to, address[] tokens, ...",8586612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3424,47399463,1741780022,0x653a4cfc9b5bccc15b836f1d6f1ae5759cf392098bc2...,327,0x1052a007765d7c09d6a9d71185bd729f8c2e064820d8...,0xdc5d112d1fea340f3b7097d4c094971dccb271c0,0x236489bdbf9f4f297a214d365be8f7238fc247ae,0x77c207739abeb3dd3380352f52c607a08e57d837,94,KLK Genesis,...,0,81,119998,1010000000,74750,7926858,deprecated,0x42842e0e,"safeTransferFrom(address from, address to, uin...",8439769
3425,47399467,1741780034,0x5e70e57568bafd4b06220520ec5ca0704888619176f8...,328,0x954b5bf7c78f7dcb9237771cdabc2f998a388ee34c37...,0xdc5d112d1fea340f3b7097d4c094971dccb271c0,0x236489bdbf9f4f297a214d365be8f7238fc247ae,0x77c207739abeb3dd3380352f52c607a08e57d837,27,KLK Genesis,...,0,58,161353,1100000001,97344,5940584,deprecated,0x42842e0e,"safeTransferFrom(address from, address to, uin...",8439765
3426,47399471,1741780046,0x78f931c04729e892513acaf4e3083f3021c7f59ddf7a...,329,0x95ab0975b427e9e677178c75184a611ae3ef8374fe09...,0xdc5d112d1fea340f3b7097d4c094971dccb271c0,0x236489bdbf9f4f297a214d365be8f7238fc247ae,0x77c207739abeb3dd3380352f52c607a08e57d837,46,KLK Genesis,...,0,44,145728,1210000000,87050,3627575,deprecated,0x42842e0e,"safeTransferFrom(address from, address to, uin...",8439761
3427,47404574,1741795355,0x35ddef2a8bd622f9cd6fdb7ec26b48c24444500ff81e...,5,0x06300dff1aa28f7b208bee89e45d78354398d1b32f6a...,0x534a5623e901fc32ecdc92ca3c45f24358a68441,0x236489bdbf9f4f297a214d365be8f7238fc247ae,0x77c207739abeb3dd3380352f52c607a08e57d837,20471,KLK Genesis,...,0,55,145746,2020000000,87062,4311945,deprecated,0x42842e0e,"safeTransferFrom(address from, address to, uin...",8434658


In [183]:
from_addresses = df_NFT_Transfers["from"].unique()
print("参与集卡的地址数量", len(from_addresses))

参与集卡的地址数量 117


In [184]:
total_nfts = df_NFT_Transfers["tokenID"].nunique()
print("总共汇集NFT的数量：", total_nfts)

总共汇集NFT的数量： 3429


In [176]:
# 统计真NFT的哈希
real_nft_count = len(df_NFT_Transfers[df_NFT_Transfers["contractAddress"] == "0x236489bdbf9f4f297a214d365be8f7238fc247ae"])
print(real_nft_count)
print("记录总数不变，可见集卡参与者发的全是真的NFT，不存在发错NFT的情况！参与者首先第一步全都有效！")

3429
记录总数不变，可见集卡参与者发的全是真的NFT，不存在发错NFT的情况！参与者首先第一步全都有效！


In [127]:
def get_NFT_Details(chainid, contract_address, token_id):
    headers = {
        "User-Agent": UserAgent().random,
    }

    match chainid:
        case 56:  # BSC主网
            url = f'https://bscscan.com/nft/{contract_address}/{token_id}'
        case 1:  # Ethereum主网
            url = f'https://etherscan.io/nft/{contract_address}/{token_id}'
        case _:
            raise ValueError("Unsupported chain ID")  
          
    response = requests.get(
        url,
        headers=headers
    )
    if response.status_code != 200:
        raise Exception(f"Failed to fetch NFT details: {response.status_code}")
    # 解析HTML
    tree = etree.HTML(response.text)

    info = {
        "owner_address": tree.xpath('normalize-space(//*[@id="ContentPlaceHolder1_divOwner"]/div[2]/div/a[1]/text())'),
        "contract_address": contract_address,
        "creator_address": tree.xpath('normalize-space(//*[@id="collapseDetails"]/div/div[3]/div[2]/div/a[1]/text())'),
        "token_id": token_id,
        "token_standard": tree.xpath('normalize-space(//*[@id="collapseDetails"]/div/div[6]/div[2]/text())'),
        "properties": tree.xpath('normalize-space(//*[@id="collapseProperties"]/div/div/div/div/p/text())'),
    }

    return info



In [9]:
# 获取合约的ABI
def get_abi(chainid, address):
    # "0x26d85a13212433fe6a8381969c2b0db390a0b0ae"
    params = {
    "chainid": chainid,  # BSC主网
    "module": "contract",
    "action": "getabi",  # 获取BEP-721/BEP-1155转账记录
    "address": address,
    "apikey": API_KEY,
}
    data = query(params)
    if not data:
        print("无交易记录或获取失败")
        return []
    return json.loads(data)

def fetch_metadata(token_id, contract, session):
    """获取单个 token 的元数据"""
    try:
        token_uri = contract.functions.tokenURI(token_id).call()
        if token_uri.startswith("ipfs://"):
            token_uri = token_uri.replace("ipfs://", "https://ipfs.io/ipfs/")
        with session.get(token_uri, timeout=10) as response:
            response.raise_for_status()
            metadata = response.json()
            return token_id, metadata.get("attributes", [])
    except Exception as e:
        print(f"处理 token ID {token_id} 失败: {e}")
        return token_id, None
    


In [199]:
def get_nft_inventory():
    params = {
        "module": "account",
        "action": "tokenholderlist",
        "contractaddress": "0x236489bdbf9f4f297a214d365be8f7238fc247ae",
        "page": 1,
        "offset": 100,  # 每页100条记录
        "apikey": API_KEY
    }
    data = query(params)
    if not data:
        print("无NFT持有者记录或获取失败")
        return []
    return data



## 统计哪些地址参与了集卡

In [122]:
from web3 import Web3
import requests
import json
from fake_useragent import UserAgent
from concurrent.futures import ThreadPoolExecutor, as_completed

ua = UserAgent()
# 初始化 Web3 和合约
rpc = "https://bsc-dataseed.binance.org/"
w3 = Web3(Web3.HTTPProvider(rpc))

chainid = "56"
contract_address = "0x236489bdbf9f4f297a214d365be8f7238fc247ae"
address = w3.to_checksum_address(contract_address)
abi = get_abi(chainid, contract_address) 
contract = w3.eth.contract(address=address, abi=abi)

In [44]:
# 单个 token_id 的处理逻辑
def process_token(token_id):
    headers={"User-Agent": ua.random}

    try:
        token_uri = contract.functions.tokenURI(token_id).call()
        res = requests.get(token_uri, headers=headers,)

        if res.status_code == 200:
            metadata = res.json()
            attributes = metadata.get("attributes", [])
            if isinstance(attributes, list) and len(attributes) > 0:
                attribute = attributes[0].get("value", "Unknown")
            else:
                attribute = None
            print(f"[{token_id}] ✅ {attribute}")
        else:
            print(f"[{token_id}] ❌ Failed: {res.status_code}")
            attribute = None
    except Exception as e:
        print(f"[{token_id}] ⚠️ Error: {str(e)}")
        attribute = None

    return attribute

In [ ]:
df_NFT_Transfers["attribute"] = df_NFT_Transfers["tokenID"].apply(lambda x: process_token(int(x)))

In [41]:
df_NFT_Transfers.to_csv("df_NFT_Transfers.csv", index=False)

In [73]:
df_NFT_Transfers = pd.read_csv("df_NFT_Transfers.csv")

In [74]:
df_NFT_Transfers

,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,tokenID,tokenName,...,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,input,methodId,functionName,confirmations,attribute
0,47252620,1741339463,0x3853fa818918c9e94701372bfa5778db46d56e5e7ad8...,155,0xe67af1a9d0bc13464901556c1b4afac0a9c255843d8f...,0xccddd8403bf28ccdedce93c1985032281670cf48,0x236489bdbf9f4f297a214d365be8f7238fc247ae,0x77c207739abeb3dd3380352f52c607a08e57d837,99301,KLK Genesis,...,1,7768662,3000000000,4319495,4347087,deprecated,0x89d56c84,"transferERC721s(address to, address[] tokens, ...",8586612,EUR
1,47252620,1741339463,0x3853fa818918c9e94701372bfa5778db46d56e5e7ad8...,155,0xe67af1a9d0bc13464901556c1b4afac0a9c255843d8f...,0xccddd8403bf28ccdedce93c1985032281670cf48,0x236489bdbf9f4f297a214d365be8f7238fc247ae,0x77c207739abeb3dd3380352f52c607a08e57d837,99170,KLK Genesis,...,1,7768662,3000000000,4319495,4347087,deprecated,0x89d56c84,"transferERC721s(address to, address[] tokens, ...",8586612,EUR
2,47252620,1741339463,0x3853fa818918c9e94701372bfa5778db46d56e5e7ad8...,155,0xe67af1a9d0bc13464901556c1b4afac0a9c255843d8f...,0xccddd8403bf28ccdedce93c1985032281670cf48,0x236489bdbf9f4f297a214d365be8f7238fc247ae,0x77c207739abeb3dd3380352f52c607a08e57d837,20270,KLK Genesis,...,1,7768662,3000000000,4319495,4347087,deprecated,0x89d56c84,"transferERC721s(address to, address[] tokens, ...",8586612,EUR
3,47252620,1741339463,0x3853fa818918c9e94701372bfa5778db46d56e5e7ad8...,155,0xe67af1a9d0bc13464901556c1b4afac0a9c255843d8f...,0xccddd8403bf28ccdedce93c1985032281670cf48,0x236489bdbf9f4f297a214d365be8f7238fc247ae,0x77c207739abeb3dd3380352f52c607a08e57d837,24012,KLK Genesis,...,1,7768662,3000000000,4319495,4347087,deprecated,0x89d56c84,"transferERC721s(address to, address[] tokens, ...",8586612,EUR
4,47252620,1741339463,0x3853fa818918c9e94701372bfa5778db46d56e5e7ad8...,155,0xe67af1a9d0bc13464901556c1b4afac0a9c255843d8f...,0xccddd8403bf28ccdedce93c1985032281670cf48,0x236489bdbf9f4f297a214d365be8f7238fc247ae,0x77c207739abeb3dd3380352f52c607a08e57d837,21999,KLK Genesis,...,1,7768662,3000000000,4319495,4347087,deprecated,0x89d56c84,"transferERC721s(address to, address[] tokens, ...",8586612,EUR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3424,47399463,1741780022,0x653a4cfc9b5bccc15b836f1d6f1ae5759cf392098bc2...,327,0x1052a007765d7c09d6a9d71185bd729f8c2e064820d8...,0xdc5d112d1fea340f3b7097d4c094971dccb271c0,0x236489bdbf9f4f297a214d365be8f7238fc247ae,0x77c207739abeb3dd3380352f52c607a08e57d837,94,KLK Genesis,...,81,119998,1010000000,74750,7926858,deprecated,0x42842e0e,"safeTransferFrom(address from, address to, uin...",8439769,EUR
3425,47399467,1741780034,0x5e70e57568bafd4b06220520ec5ca0704888619176f8...,328,0x954b5bf7c78f7dcb9237771cdabc2f998a388ee34c37...,0xdc5d112d1fea340f3b7097d4c094971dccb271c0,0x236489bdbf9f4f297a214d365be8f7238fc247ae,0x77c207739abeb3dd3380352f52c607a08e57d837,27,KLK Genesis,...,58,161353,1100000001,97344,5940584,deprecated,0x42842e0e,"safeTransferFrom(address from, address to, uin...",8439765,USD
3426,47399471,1741780046,0x78f931c04729e892513acaf4e3083f3021c7f59ddf7a...,329,0x95ab0975b427e9e677178c75184a611ae3ef8374fe09...,0xdc5d112d1fea340f3b7097d4c094971dccb271c0,0x236489bdbf9f4f297a214d365be8f7238fc247ae,0x77c207739abeb3dd3380352f52c607a08e57d837,46,KLK Genesis,...,44,145728,1210000000,87050,3627575,deprecated,0x42842e0e,"safeTransferFrom(address from, address to, uin...",8439761,EUR
3427,47404574,1741795355,0x35ddef2a8bd622f9cd6fdb7ec26b48c24444500ff81e...,5,0x06300dff1aa28f7b208bee89e45d78354398d1b32f6a...,0x534a5623e901fc32ecdc92ca3c45f24358a68441,0x236489bdbf9f4f297a214d365be8f7238fc247ae,0x77c207739abeb3dd3380352f52c607a08e57d837,20471,KLK Genesis,...,55,145746,2020000000,87062,4311945,deprecated,0x42842e0e,"safeTransferFrom(address from, address to, uin...",8434658,GBP


## 集卡地址应该收到多少空投

In [ ]:
# 空投玩法：
    # 五种属性 KLK、AED、EUR、GBP、USD
    # 任意 ID 的 KLK 属性 和 成套的 ID > 20250 的其他属性NFT（简称普卡）
    # ID <= 20250 的非 KLK 属性的 NFT 称为白卡，只能参与 1:10 兑换 $KLK
    # 参与集卡空投的 NFT 将不计算在 1:10 兑换的 $KLK，空投领取以地址当前存在的 NFT 数量为准
# 空投计算规则：
    # 1*KLK + 1*(AED+EUR+GBP+USD) = 1000 $KLK
    # 1*KLK + 3*(AED+EUR+GBP+USD) = 2000 $KLK
    # 1*KLK + 5*(AED+EUR+GBP+USD) = 3000 $KLK
    # 1*KLK + 8*(AED+EUR+GBP+USD) = 4000 $KLK
    # 1*KLK + 12*(AED+EUR+GBP+USD) = 6000 $KLK
    # 1*KLK + 20*(AED+EUR+GBP+USD) = 8000 $KLK
    # 1*KLK + 25*(AED+EUR+GBP+USD) = 10000 $KLK
def airdrop(row):
    if (row["KLK"] >= 1) and (row["AED"] >= 25) and (row["EUR"] >= 25) and (row["GBP"] >= 25) and (row["USD"] >= 25):
        return 10000
    elif (row["KLK"] >= 1) and (row["AED"] >= 20) and (row["EUR"] >= 20) and (row["GBP"] >= 20) and (row["USD"] >= 20):
        return 8000
    elif (row["KLK"] >= 1) and (row["AED"] >= 12) and (row["EUR"] >= 12) and (row["GBP"] >= 12) and (row["USD"] >= 12):
        return 6000
    elif (row["KLK"] >= 1) and (row["AED"] >= 8) and (row["EUR"] >= 8) and (row["GBP"] >= 8) and (row["USD"] >= 8):
        return 4000
    elif (row["KLK"] >= 1) and (row["AED"] >= 5) and (row["EUR"] >= 5) and (row["GBP"] >= 5) and (row["USD"] >= 5):
        return 3000
    elif (row["KLK"] >= 1) and (row["AED"] >= 3) and (row["EUR"] >= 3) and (row["GBP"] >= 3) and (row["USD"] >= 3): 
        return 2000
    elif (row["KLK"] >= 1) and (row["AED"] >= 1) and (row["EUR"] >= 1) and (row["GBP"] >= 1) and (row["USD"] >= 1):
        return 1000

df_nft_reward = df_NFT_Transfers[(df_NFT_Transfers["tokenID"].astype(int) % 1000 == 0) | (df_NFT_Transfers["tokenID"].astype(int) >= 20250)].groupby(["from", "attribute"]).size().unstack(fill_value=0)
df_nft_reward = df_nft_reward[~(df_nft_reward == 0).any(axis=1)]
df_nft_reward["aidrop"] = df_nft_reward.apply(lambda row: airdrop(row), axis=1)
df_nft_reward


attribute,AED,EUR,GBP,KLK,USD,aidrop
from,,,,,,
0x017e7ae19213718e0f909703619870e1b3c3da26,12,12,12,1,12,6000
0x02339445dc7e52ff418daf63083a547ed199932e,12,12,12,1,12,6000
0x033f0c12e8897329550281300c1abef653aa1751,12,12,12,1,12,6000
0x03465fc46b6ef138c60bcc74f587621b23e997b3,12,12,12,1,12,6000
0x09096eaf1158c1a15f469b20390c709a1c13d9de,5,5,5,1,5,3000
...,...,...,...,...,...,...
0xf3b6b294d576c5d9bae033e658edf9efcfadc5bd,12,12,12,1,12,6000
0xf5ed1d57c808e4b5c72999a49ac215491f8c62c1,12,12,12,1,12,6000
0xf80a66193cef8f26f06045b2ee4026f20f106b9d,5,5,5,1,5,3000


In [76]:
# 总计应当空投给NFT集卡地址的空投总数
df_nft_reward.aidrop.sum()

419000

## 集卡地址究竟收到了多少空投

In [134]:
# 查询KLKO代币精度
from web3 import Web3
from fake_useragent import UserAgent

ua = UserAgent()
# 初始化 Web3 和合约
rpc = "https://bsc-dataseed.binance.org/"
w3 = Web3(Web3.HTTPProvider(rpc))

chainid = "56"
contract_address = "0x215324115417a13879f865dc5bb651b18a781a59"
address = w3.to_checksum_address(contract_address)
abi = get_abi(chainid, contract_address) 
contract = w3.eth.contract(address=address, abi=abi)

decimals = contract.functions.decimals().call()

# 获取精度
print("Decimals:", decimals)


Decimals: 18


In [135]:
def get_ERC20_Transfers(chainid, address, contractaddress=""):
    params = {
    "chainid": chainid, 
    "module": "account",
    "action": "tokentx",
    "contractaddress":contractaddress,  
    "address": address,
    "startblock": 0,
    "endblock": 99999999,
    "sort": "asc",
    "apikey": API_KEY,
    "page":0, 
    "offset": 100 
}
    data = query(params)
    if not data:
        return []
    return data


In [172]:
def reward(row):
    from_address = "0xA9A4a1B1EAf7978d64136cdA0Be84Da223243f35" # KLKO空投发放者
    to_address = row.name
    data = get_ERC20_Transfers(chainid="56", contractaddress="0x215324115417a13879f865dc5bb651b18a781a59", address=to_address)
    print(f"Processing address: {to_address}, from address: {from_address}")
    if not data:
        return 0

    df_reward = pd.DataFrame(data)
    df_reward = df_reward[
    (df_reward["from"].str.lower() == from_address.lower()) &
    (df_reward["to"].str.lower() == to_address.lower()) &
    (df_reward["functionName"] == 'transfer(address dst, uint256 amount)')
    ]

    if df_reward.empty:
        return 0
    
    return (df_reward["value"].astype(float) / (10 ** 18)).sum()
  
# klko非claim数量
klko_amount_not_claim = df_nft_reward.apply(lambda row: reward(row), axis=1)


Processing address: 0x017e7ae19213718e0f909703619870e1b3c3da26, from address: 0xA9A4a1B1EAf7978d64136cdA0Be84Da223243f35
Processing address: 0x02339445dc7e52ff418daf63083a547ed199932e, from address: 0xA9A4a1B1EAf7978d64136cdA0Be84Da223243f35
Processing address: 0x033f0c12e8897329550281300c1abef653aa1751, from address: 0xA9A4a1B1EAf7978d64136cdA0Be84Da223243f35
Processing address: 0x03465fc46b6ef138c60bcc74f587621b23e997b3, from address: 0xA9A4a1B1EAf7978d64136cdA0Be84Da223243f35
Processing address: 0x09096eaf1158c1a15f469b20390c709a1c13d9de, from address: 0xA9A4a1B1EAf7978d64136cdA0Be84Da223243f35
Processing address: 0x0ac38240d728af03c753c87fdf8c85b57da51e17, from address: 0xA9A4a1B1EAf7978d64136cdA0Be84Da223243f35
Processing address: 0x13b8e0be36041a42347c7378a2e37387bbb3df60, from address: 0xA9A4a1B1EAf7978d64136cdA0Be84Da223243f35
Processing address: 0x167f73eadefce305013a00a3b4b19c956643dbc1, from address: 0xA9A4a1B1EAf7978d64136cdA0Be84Da223243f35
Processing address: 0x16d2077fb5

In [177]:
df_nft_reward["airdrop_recived"] = klko_amount_not_claim

In [178]:
df_nft_reward

attribute,AED,EUR,GBP,KLK,USD,aidrop,airdrop_recived
from,,,,,,,
0x017e7ae19213718e0f909703619870e1b3c3da26,12,12,12,1,12,6000,0.0
0x02339445dc7e52ff418daf63083a547ed199932e,12,12,12,1,12,6000,0.0
0x033f0c12e8897329550281300c1abef653aa1751,12,12,12,1,12,6000,0.0
0x03465fc46b6ef138c60bcc74f587621b23e997b3,12,12,12,1,12,6000,6000.0
0x09096eaf1158c1a15f469b20390c709a1c13d9de,5,5,5,1,5,3000,0.0
...,...,...,...,...,...,...,...
0xf3b6b294d576c5d9bae033e658edf9efcfadc5bd,12,12,12,1,12,6000,6000.0
0xf5ed1d57c808e4b5c72999a49ac215491f8c62c1,12,12,12,1,12,6000,0.0
0xf80a66193cef8f26f06045b2ee4026f20f106b9d,5,5,5,1,5,3000,0.0


In [179]:
df_nft_reward.to_csv("df_nft_reward.csv", index=False)

In [182]:
print("集卡地址理应收到的空投数量：", int(df_nft_reward.aidrop.sum()))
print("集卡地址实际收到的空投数量：", int(df_nft_reward.airdrop_recived.sum()))

集卡地址理应收到的空投数量： 419000
集卡地址实际收到的空投数量： 56000
